## Deliverable 3. Create a Travel Itinerary Map.

In [41]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy
import gmaps.datasets

# Import API key
from config import keys
google_key = keys["google_maps"]

# Configure gmaps
gmaps.configure(api_key=google_key)

In [42]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("WeatherPy_vacation.csv")
vacation_df

,Unnamed: 0,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
0,1,Isangel,VU,74.05,-19.5500,169.2667,overcast clouds,Tanna Lodge
1,3,Kudat,MY,77.99,6.8837,116.8477,overcast clouds,Dream Garden Hotel
2,4,Deogarh,IN,67.84,21.5333,84.7333,clear sky,Amit Henej House
3,5,Rincon,US,66.16,32.2960,-81.2354,broken clouds,Days Inn Rincon
4,6,Vao,NC,68.70,-22.6667,167.4833,overcast clouds,Hotel Kou-Bugny
...,...,...,...,...,...,...,...,...
313,675,Uyuni,BO,67.10,-20.4597,-66.8250,scattered clouds,Hostal La Magia de Uyuni
314,677,Cizre,TR,69.39,37.3272,42.1903,clear sky,Divan Otel Cizre / Şırnak
315,678,Biloela,AU,62.94,-24.4167,150.5000,broken clouds,Biloela Silo Motor Inn
316,679,Miri,MY,77.14,4.4148,114.0089,light rain,"Mega Hotel, Miri"


In [43]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather description</dt><dd>{Description}</dd>
</dl>

"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [44]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
gmaps.configure(api_key=google_key)
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] == "Laguna"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Laguna"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Half Moon Bay"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Pacific Grove"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Ridgecrest"] 

In [46]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = (float(vacation_start["Lat"]),float(vacation_start["Lng"]))
end = (float(vacation_end["Lat"]),float(vacation_end["Lng"]))
stop1 = (float(vacation_stop1["Lat"]),float(vacation_stop1["Lng"]))
stop2 = (float(vacation_stop2["Lat"]),float(vacation_stop2["Lng"]))
stop3 = (float(vacation_stop3["Lat"]),float(vacation_stop3["Lng"]))


In [47]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
gmaps.configure(api_key=google_key)

fig = gmaps.figure()
road_map = gmaps.directions_layer(start,end,travel_mode = "DRIVING",waypoints = [stop1,stop2,stop3])
fig.add_layer(road_map)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [48]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,Unnamed: 0,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
0,322,Laguna,US,70.11,38.4210,-121.4238,clear sky,Holiday Inn Express & Suites Elk Grove Central...
1,161,Half Moon Bay,US,68.05,37.4636,-122.4286,mist,"The Ritz-Carlton, Half Moon Bay"
2,147,Pacific Grove,US,67.95,36.6177,-121.9166,clear sky,Lovers Point Inn
3,495,Ridgecrest,US,77.02,35.6225,-117.6709,clear sky,SpringHill Suites by Marriott Ridgecrest


In [37]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather description</dt><dd>{Description} {Max Temp} °F</dd>
</dl>

"""
# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map.
gmaps.configure(api_key=google_key)

fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))